In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, InputSpec
import os
import warnings

# Custom callback to stop training when validation accuracy reaches 100%
class EarlyStoppingAtAccuracy(Callback):
    def __init__(self, monitor='val_accuracy', value=1.0, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn(f'Early stopping requires {self.monitor} available!', RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print(f'\nEpoch {epoch + 1}: Validation accuracy reached {self.value * 100:.2f}%, stopping training.')
            self.model.stop_training = True

# Define SE Block
class SEBlock(Layer):
    def __init__(self, reduction=16, **kwargs):
        super(SEBlock, self).__init__(**kwargs)
        self.reduction = reduction

    def build(self, input_shape):
        self.input_spec = InputSpec(ndim=4)
        self.channels = input_shape[-1]
        self.global_avg_pool = GlobalAveragePooling2D()
        self.dense1 = Dense(self.channels // self.reduction, activation='relu', kernel_initializer='he_normal', use_bias=False)
        self.dense2 = Dense(self.channels, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)
        super(SEBlock, self).build(input_shape)

    def call(self, inputs):
        se = self.global_avg_pool(inputs)
        se = tf.expand_dims(tf.expand_dims(se, 1), 1)
        se = self.dense1(se)
        se = self.dense2(se)
        return inputs * se

# Set dataset directory and parameters
dataset_dir = "C:/Users/NAMITHAA/Downloads/archive/AID"
target_size = (224, 224)  # Target size for DenseNet121
batch_size = 32

# Define the data generators with increased augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=45,  # Increased rotation range
    width_shift_range=0.2,  # Match width shift
    height_shift_range=0.2,  # Match height shift
    shear_range=0.2,  # Match shear range
    zoom_range=0.2,  # Match zoom range
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Use 20% of the data for validation
)

# Data generators for training and validation
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Set as training data
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

# Load DenseNet121 with pre-trained ImageNet weights
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of DenseNet121
x = base_model.output
x = SEBlock()(x)  # Insert SE block
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)  # Match number of units
x = Dropout(0.3)(x)  # Match dropout rate
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Fine-tune more layers of DenseNet121
for layer in base_model.layers[:-50]:
    layer.trainable = False  # Match number of frozen layers
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Compile the model with a lower learning rate and learning rate scheduler
optimizer = Adam(learning_rate=0.0001)  # Using Adam optimizer
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
filepath = "C:/Users/NAMITHAA/Downloads/save_models.keras"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.000001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
# Add custom callback to stop training when validation accuracy reaches 100%
early_stopping_accuracy = EarlyStoppingAtAccuracy(monitor='val_accuracy', value=1.0, verbose=1)

# Train the model with more epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=40,  # Match the number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[checkpoint, early_stopping, reduce_lr, early_stopping_accuracy],
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')


Found 8000 images belonging to 30 classes.
Found 2000 images belonging to 30 classes.
Epoch 1/40


C:\Users\NAMITHAA\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2488 - loss: 2.8286
Epoch 1: val_accuracy improved from -inf to 0.75302, saving model to C:/Users/NAMITHAA/Downloads/save_models.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 1052s 4s/step - accuracy: 0.2495 - loss: 2.8260 - val_accuracy: 0.7530 - val_loss: 1.0695 - learning_rate: 1.0000e-04
Epoch 2/40


c:\Program Files\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)



Epoch 2: val_accuracy improved from 0.75302 to 0.81250, saving model to C:/Users/NAMITHAA/Downloads/save_models.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8125 - val_loss: 0.9862 - learning_rate: 1.0000e-04
Epoch 3/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7473 - loss: 0.9822
Epoch 3: val_accuracy improved from 0.81250 to 0.86038, saving model to C:/Users/NAMITHAA/Downloads/save_models.keras
250/250 ━━━━━━━━━━━━━━━━━━━━ 412s 2s/step - accuracy: 0.7474 - loss: 0.9816 - val_accuracy: 0.8604 - val_loss: 0.5466 - learning_rate: 1.0000e-04
Epoch 4/40

Epoch 4: val_accuracy did not improve from 0.86038
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8125 - val_loss: 0.8317 - learning_rate: 1.0000e-04
Epoch 5/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8416 - loss: 0.5836
Epoch 5: val_accuracy improved from 0.86038 to 0.89062, saving model to C:/Us